# AI 전문가 교육과정 실습 2 - part Final

***
### NLP응용: 토픽 추출
Applied Natrual Language Processing: Topic Modeling

강사: 차미영 교수 (카이스트 전산학부)    
조교: 신민기, 정현규 (카이스트 전산학부)

# LDA on Korean Data

In [1]:
!pip install pyLDAvis==3.2.2
!pip install konlpy 

In [2]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


  Using cached JPype1-py3-0.5.5.4.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [37 lines of output]
      
      ********* DEPRECATION WARNING *********
      ********* DEPRECATION WARNING *********
      
      Traceback (most recent call last):
        File "C:\Users\AI_15\AppData\Local\Temp\pip-install-a7ihv65v\jpype1-py3_90055769632145df87a93e5951694e72\setup.py", line 483, in <module>
          config = WindowsJDKFinder()
        File "C:\Users\AI_15\AppData\Local\Temp\pip-install-a7ihv65v\jpype1-py3_90055769632145df87a93e5951694e72\setup.py", line 206, in __init__
          java_home = self.find_jdk_home()
        File "C:\Users\AI_15\AppData\Local\Temp\pip-install-a7ihv65v\jpype1-py3_90055769632145df87a93e5951694e72\setup.py", line 254, in find_jdk_home
          raise NoJDKError(visited_folders)
      __main__.NoJDKError: No JDK found
      
      During handling of the above exception, another exception occurred:
      
      Traceback (

In [3]:
import glob
import pandas as pd
import matplotlib.pyplot as plt

import re
##from konlpy.tag import Mecab
from konlpy.tag import Okt
from wordcloud import WordCloud

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim 
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings(action='ignore')

c:\Users\AI_15\AppData\Local\Programs\Python\Python38\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [4]:
from konlpy.tag import Okt
#okt = Okt()
import platform
print(platform.architecture())
#print(okt.pos(u'이것도 되나욬?ㅋㅋ', norm=True, stem=True))

('64bit', 'WindowsPE')


In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AI_15\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AI_15\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AI_15\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
#%cd drive/My\ Drive

# Data load

In [7]:
file_list = glob.glob("./data/*.json")

In [8]:
# total_df = None
# first = True
# for file_name in file_list:
#     print(file_name)
#     if first:   
#         total_df = pd.read_json(file_name, lines=True)
#         first = False
#     else:
#         temp = pd.read_json(file_name, lines=True)
#         total_df = pd.concat([total_df, temp])
# total_df.reset_index(inplace = True)
# original_df = total_df.copy()

In [9]:
total_df = pd.read_json(file_list[0], lines=True)
total_df.reset_index(inplace = True)
original_df = total_df.copy()

In [10]:
total_df.head(3)

,index,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1241514909659086849,1241514909659086848,2020-03-21 23:59:55,2020-03-22,08:59:55,¢¥eCN©öI¡¾©ö C¡ÍA¨ª¨öA,1239857975398817792,ch10v3,천사님 모시는 ⑧,...,,,,,,"[{'user_id': '1239857975398817792', 'username'...",,,,
1,1,1241514884665184257,1241514884665184256,2020-03-21 23:59:49,2020-03-22,08:59:49,¢¥eCN©öI¡¾©ö C¡ÍA¨ª¨öA,158630288,seaok,GOD BLESS YOU,...,,,,,,"[{'user_id': '158630288', 'username': 'seaok'}]",,,,
2,2,1241514882362675204,1241475915298766848,2020-03-21 23:59:48,2020-03-22,08:59:48,¢¥eCN©öI¡¾©ö C¡ÍA¨ª¨öA,1138535280380719104,lenallurekr,레나,...,,,,,,"[{'user_id': '1138535280380719104', 'username'...",,,,


In [11]:
total_df.describe()

,index,id,conversation_id,user_id,replies_count,retweets_count,likes_count,video
count,20069.000000,2.006900e+04,2.006900e+04,2.006900e+04,20069.000000,20069.00000,20069.000000,20069.000000
mean,10034.000000,1.241306e+18,1.241103e+18,6.051826e+17,0.467039,8.06094,5.688774,0.010015
std,5793.565612,8.862836e+13,4.615141e+15,5.493509e+17,2.448691,138.42203,61.655466,0.099577
min,0.000000,1.241153e+18,9.818432e+17,8.150500e+05,0.000000,0.00000,0.000000,0.000000
25%,5017.000000,1.241233e+18,1.241219e+18,4.891601e+08,0.000000,0.00000,0.000000,0.000000
50%,10034.000000,1.241302e+18,1.241290e+18,8.412471e+17,0.000000,0.00000,0.000000,0.000000
75%,15051.000000,1.241370e+18,1.241361e+18,1.154752e+18,1.000000,0.00000,1.000000,0.000000
max,20068.000000,1.241515e+18,1.241515e+18,1.241490e+18,144.000000,10116.00000,3266.000000,1.000000


# Pre-processing function

In [12]:
# Basic Cleaning Text Function
def CleanText(readData):

    # Remove Retweets 
    text = re.sub('RT @[\w_]+: ', '', readData)

    # Remove Mentions
    text = re.sub('@[\w_]+', '', text)

    # Remove or Replace URL 
    # text = url_re.sub('URL', text)
    text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", ' ', text) # start with http
    text = re.sub(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", ' ', text) # Don't start with http

    # Remove Hashtag
    text = re.sub('[#]+[0-9a-zA-Z_]+', ' ', text)

    # Remove Garbage Words (ex. &lt, &gt, etc)
    text = re.sub('[&]+[a-z]+', ' ', text)

    # Remove Special Characters
    text = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣]', ' ', text)

    # Remove Numbers (If you want, activate the code)
    text = re.sub(r'\d+',' ',text)

    # Remove English (If you want, activate the code)
    text = re.sub('[a-zA-Z]' , ' ', text)

    # Remove newline
    text = text.replace('\n',' ')

    # Remove multi spacing & Reform sentence
    text = ' '.join(text.split())

    return text

In [ ]:
def preprocessing_okt(readData):
    #### Clean text
    sentence = CleanText(readData)

    #### Tokenize
    morphs = Okt.pos(sentence)

    JOSA = ["Josa"]
    SIGN = ["SF", "SE", "SSO", "SSC", "SC", "SY"]
    TERMINATION = ["EP", "EF", "EC", "ETN", "ETM"] # 어미
    SUPPORT_VERB = ["VX"]
    NUMBER = ["SN"]

    # Remove JOSA, EOMI, etc
    morphs[:] = (morph for morph in morphs if morph[1] not in JOSA+SIGN+TERMINATION+SUPPORT_VERB)


In [13]:
def preprocessing_mecab(readData):
    #### Clean text
    sentence = CleanText(readData)

    #### Tokenize
    morphs = mecab.pos(sentence)

    JOSA = ["JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ", "JX", "JC"]
    SIGN = ["SF", "SE", "SSO", "SSC", "SC", "SY"]
    TERMINATION = ["EP", "EF", "EC", "ETN", "ETM"] # 어미
    SUPPORT_VERB = ["VX"]
    NUMBER = ["SN"]

    # Remove JOSA, EOMI, etc
    morphs[:] = (morph for morph in morphs if morph[1] not in JOSA+SIGN+TERMINATION+SUPPORT_VERB)

    # If you want to save only Nouns:
    # morphs = mecab.nouns(sentence)

    # Remove Stopwords
    morphs[:] = (morph for morph in morphs if morph[0] not in korean_stopwords["형태"].tolist())
    morphs[:] = (morph for morph in morphs if morph[0] not in my_korean_stopwords["형태"].tolist())

    # Remove length-1 words  
    morphs[:] = (morph for morph in morphs if not (len(morph[0]) == 1))

    # Remove Numbers
    morphs[:] = (morph for morph in morphs if morph[1] not in NUMBER)

    # Result pop-up
    result = []
    for morph in morphs:
        result.append(morph[0])

    return result

In [14]:
# Korean Stopwords Load
korean_stopwords = pd.read_csv("./data/korean_stopwords.txt", delimiter='\t', names=["형태", "품사", "비율"])

# Add Custom Korean Stopwords 
my_data = [["님", "NNG"], ["들", "XSN"], ["ㅋㅋㄱㅋㄱㅋ", "NNG"], 
           ["오늘", "NNG"], ["얘기", "NNG"], ["ㅠㅠ", "NNG"], ["없이", ], ["딱히", ], 
           ['ㅋㅋ', ], ['ㅋㅋㅋ', ], ["그런데", ], ["누구", ], ["여기저기", ]]

my_korean_stopwords = pd.DataFrame(my_data, columns = ['형태', '품사'])

# Main 
#mecab = Mecab(dicpath="/usr/local/lib/mecab/dic/mecab-ko-dic") # Mecab Dictionary Path

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [ ]:
SAMPLE_TEXT = "RT @boxplus01: 美언론, 'abc한국 코로나 확산주범은 신천지와 보수세력' https://t.co/Phq0l48aUm"
print("Before preprocessing : {}".format(SAMPLE_TEXT))
print("After preprocessing : {}".format(preprocessing_mecab(SAMPLE_TEXT)))

In [ ]:
total_df['tweet'] = total_df['tweet'].apply(lambda x: preprocessing_mecab(x))

In [ ]:
total_df['tweet']

# # Find topics

In [ ]:
data_lemmatized = total_df['tweet'].tolist()

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=2,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualization

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

# Choose the number of topics

In [ ]:
from tqdm import tqdm

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step)):
        model =  gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=2,
                                           alpha='auto',
                                           per_word_topics=True)
        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=35, step=6)

In [ ]:
limit=35; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

# Finding the dominant topic in each sentence

In [ ]:
optimal_model = model_list[5]

In [ ]:
optimal_model.per_word_topics = False

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', optimal_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=optimal_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=None):
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(tqdm(ldamodel[corpus])):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=total_df['tweet'])

In [ ]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
df_dominant_topic.head(50)